In [14]:
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import pickle
import requests
import numpy as np
import pandas as pd
import json
headers = {'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:76.0) Gecko/20100101 Firefox/76.0'}

In [33]:
search_add = 'https://www.wyzant.com/match/search?sort=1&d=20&utc_offset=-5&min_price=10&max_price=200&min_age=18&max_age=100&gender_pref=none&st=5&kw=physics&ol=false&z='
zip_code = 78750
url = search_add + str(zip_code)

chromedriver = '/home/codonnell/chromedriver.exe'
driver = webdriver.Chrome(chromedriver)
driver.get(url)

#run this, then physically enter fields to get to search page
#then reopen search url above and it should work this time
#search for all tutor links on page

In [10]:
def get_tutor_info(zip_code, links, headers):
    
    mains = []
    scheds = []

    empty = {'sunday': [None], 'monday': [None], 'tuesday': [None], 'wednesday': [None],
         'thursday': [None], 'friday': [None], 'saturday': [None]}
    
    for i, link in enumerate(links):
        r = requests.get(link, headers = headers).text
        soup = BeautifulSoup(r, 'html.parser')

        tags = soup.find_all('script')
        try:
            time_tutoring = soup.find_all('h3')[0].text
        except:
            time_tutoring = []
        bio_list = soup.find_all('p', {'class':  'spc-zero-n spc-sm-s'})
        bio = [part.text for part in bio_list]
        
        if soup.find_all('i' ,{'class':'wc-background-pass wc-green'}):
            bck = 1
            bck_passed = soup.find_all('p', {'class':"spc-zero"})[1].text
        else:
            bck = 0
            bck_passed = []
            
        edu = soup.find_all('section' ,{'class':"spc-med-s"})
        if edu:
            educ =[e.text for e in edu]
        else:
            educ = []
        
        try:
            main_details = json.loads(tags[0].string)
            main_details['time_tutoring'] = time_tutoring
            main_details['bio'] = bio
            main_details['background_check'] = bck
            main_details['date_background_passed'] = bck_passed
            main_details['education'] = educ
        
            mains.append(main_details)
            try:
                scheds.append(json.loads(tags[6].string.split('=')[1][:-1]))
            except:
                scheds.append(empty)
        except:
            pass
                
        time.sleep(np.random.rand()*5)
        
    df = pd.merge(pd.DataFrame(mains), pd.DataFrame(scheds), left_index=True, right_index=True)
    df.to_csv('/home/codonnell/tutors/tutnew/tutors_' + str(zip_code) + '.csv')

In [11]:
def save_tut_id(unique_tuts):
    with open('/home/codonnell/tutors_ids.txt', 'w') as filehandle:
        for item in unique_tuts:
            filehandle.write('%s\n' % item)

In [11]:
already_done = [77449,11368,60629,90011,79936,30044,80209,89138,28208,84102,98122,33128,
               46201, 55401,43215,19102,94108,22046,91911,37128,85001,78214,48201,63108,
               80303, 926, 8701]
pop_zips = pd.read_csv('/home/codonnell/Insight/Insight_Project_AsTutor/data/raw/zips_incomes/zips_ga_co_wa.csv')

np.setdiff1d(pop_zips['zip_code'].values,already_done)

array([30004, 30016, 30022, 30024, 30032, 30034, 30039, 30040, 30041,
       30043, 30047, 30052, 30058, 30062, 30064, 30066, 30067, 30075,
       30076, 30080, 30083, 30093, 30096, 30097, 30101, 30114, 30127,
       30134, 30135, 30144, 30157, 30188, 30236, 30253, 30263, 30281,
       30318, 30331, 30349, 30506, 30518, 30606, 30721, 30906, 30907,
       30909, 31088, 31419, 31907, 80003, 80004, 80010, 80011, 80012,
       80013, 80014, 80015, 80016, 80017, 80020, 80021, 80022, 80027,
       80031, 80111, 80122, 80123, 80126, 80127, 80128, 80134, 80138,
       80204, 80205, 80210, 80211, 80219, 80220, 80221, 80226, 80227,
       80228, 80229, 80231, 80233, 80239, 80241, 80260, 80401, 80501,
       80503, 80504, 80524, 80525, 80526, 80537, 80538, 80601, 80631,
       80634, 80906, 80909, 80911, 80916, 80917, 80918, 80920, 81001,
       81005, 81007, 81504, 98003, 98004, 98006, 98012, 98023, 98026,
       98030, 98031, 98032, 98033, 98034, 98036, 98042, 98052, 98056,
       98058, 98059,

In [11]:
zips = np.setdiff1d(pop_zips['zip_code'].values,already_done)[-7:]
#zips = already_done
subjects = ['physics', 'math', 'spanish', 'english', 'computer']
links = []
tot_tuts = []

for zip_code in zips:
    links = []
    tot_tuts = []
    
    for subject in subjects:
            
        url = 'https://www.wyzant.com/match/search?sort=1&d=20&utc_offset=-5&min_price=10&max_price=200&min_age=18&max_age=100&gender_pref=none&st=0&kw=' + subject + '&ol=false&z=' + str(zip_code)
        driver.get(url)
        
        try:
            num_tuts = driver.find_elements_by_tag_name('strong')[2].text #per subject given zip
        except:
            num_tuts = 0
        
        lks = driver.find_elements_by_xpath('.//a[@class="tutor-card flex"]')
        lks_found = [link.get_attribute('href') for link in lks]
        links.append(lks_found)
        
        try:
            if int(num_tuts.split(' ')[0]) > 10:
                extras = int(np.floor(int(num_tuts.split(' ')[0])/10)) + 1
                for i in range(1,extras):
                    urlmore = 'https://www.wyzant.com/match/search/more?d=20&gender_pref=none&kw=' + subject + '&max_age=100&max_price=500&min_age=18&min_price=10&ol=false&page_number=' + str(i) + '&sort=1&st=0&utc_offset=-5&z=' +str(zip_code)
                    driver.get(urlmore)
                    more_lks = driver.find_elements_by_xpath('.//a[@class="tutor-card flex"]')
                    more_lks_found = [link.get_attribute('href') for link in more_lks]
                    links.append(more_lks_found)
                    time.sleep(np.random.rand()*5)
            else:
                pass
        except:
            pass
            
            
        tot_tuts.append(num_tuts) #per subject given zip
            
            
        time.sleep(np.random.rand()*5)
    
    unique_tuts = np.unique(np.array([item for sublist in links for item in sublist]))
    save_tut_id(unique_tuts)
    get_tutor_info(zip_code, unique_tuts, headers)
    tut_num_df = pd.DataFrame({'zip_code': [str(zip_code)]*5, 'tot_tuts': tot_tuts})
    tut_num_df.to_csv('/home/codonnell/tutors//tutnew/tut_num_' +str(zip_code) + '.csv')
    
    time.sleep(np.random.rand()*5)

In [23]:
already_done = [77449,11368,60629,90011,79936,30044,80209,89138,28208,84102,98122,33128,
               46201, 55401,43215,19102,94108,22046,91911,37128,85001,78214,48201,63108,
               80303, 926, 8701]
#state = 'nv_nj_nm'
state = 'ol'
#pop_zips = pd.read_csv('/home/codonnell/Insight/Insight_Project_AsTutor/data/raw/zips_incomes/zips_' + state + '.csv')

#np.setdiff1d(pop_zips['zip_code'].values,already_done)

In [34]:
#zips = np.setdiff1d(pop_zips['zip_code'].values,already_done)
#zips = already_done
zips = [78751]
subjects = ['physics', 'math', 'spanish', 'english', 'computer']
links = []
tot_tuts_dfs = []

for zip_code in zips:
    tot_tuts = []
    
    for subject in subjects:
            
        url = 'https://www.wyzant.com/match/search?sort=1&d=20&utc_offset=-5&min_price=10&max_price=200&min_age=18&max_age=100&gender_pref=none&st=0&kw=' + subject + '&ol=true&z=' + str(zip_code)
        driver.get(url)
        
        try:
            num_tuts = driver.find_elements_by_tag_name('strong')[2].text #per subject given zip
        except:
            num_tuts = 0
        
        lks = driver.find_elements_by_xpath('.//a[@class="tutor-card flex"]')
        lks_found = [link.get_attribute('href') for link in lks]
        links.append(lks_found)
        
        try:
            if int(num_tuts.split(' ')[0].replace(',','')) > 10:
                extras = int(np.floor(int(num_tuts.split(' ')[0].replace(',',''))/10)) + 1
                for i in range(1,extras):
                    urlmore = 'https://www.wyzant.com/match/search/more?d=20&gender_pref=none&kw=' + subject + '&max_age=100&max_price=500&min_age=18&min_price=10&ol=true&page_number=' + str(i) + '&sort=1&st=0&utc_offset=-5&z=' +str(zip_code)
                    driver.get(urlmore)
                    more_lks = driver.find_elements_by_xpath('.//a[@class="tutor-card flex"]')
                    more_lks_found = [link.get_attribute('href') for link in more_lks]
                    links.append(more_lks_found)
                    time.sleep(np.random.rand()*5)
            else:
                pass
        except:
            pass
            
            
        tot_tuts.append(num_tuts) #per subject given zip
        time.sleep(np.random.rand()*5)
        
    tot_tuts_dfs.append(pd.DataFrame({'zip_code': [str(zip_code)]*5, 'tot_tuts': tot_tuts}))
    time.sleep(np.random.rand()*5)      
            
    
unique_tuts = np.unique(np.array([item for sublist in links for item in sublist]))
save_tut_id(unique_tuts)
get_tutor_info(state, unique_tuts, headers)
tut_num_df = pd.concat(tot_tuts_dfs)
tut_num_df.to_csv('/home/codonnell/tutors//tutnew/tut_num_' + state + '.csv')
    

SSLError: HTTPSConnectionPool(host='www.wyzant.com', port=443): Max retries exceeded with url: /match/tutor/79003150?search_id=4214d227-099e-4f46-b9ee-d5568ddd197e (Caused by SSLError(SSLError("bad handshake: SysCallError(104, 'ECONNRESET')")))

In [35]:
len(unique_tuts)

23017

In [44]:
for i in range(15,22):
    end = 1000*i
    start = 1000*(i-1)
    get_tutor_info(state + str(end), unique_tuts[start:end], headers)

KeyboardInterrupt: 

In [19]:
get_tutor_info(state, unique_tuts, headers)

In [ ]:
'https://www.wyzant.com/match/search?sort=1&d=20&utc_offset=-5&min_price=10&max_price=200&min_age=18&max_age=100&gender_pref=none&st=5&kw=physics&ol=false&z='
'https://www.wyzant.com/match/search?sort=1&d=20&utc_offset=-5&min_price=10&max_price=200&min_age=18&max_age=100&gender_pref=none&st=0&kw=' + subject + '&ol=false&z=' + str(zip_code)
'https://www.wyzant.com/match/search?sort=1&d=20&utc_offset=-5&min_price=10&max_price=200&min_age=18&max_age=100&gender_pref=none&st=0&kw=' + subject + '&ol=false&z=' + str(zip_code)
'https://www.wyzant.com/match/search?sort=1&d=20&utc_offset=-5&min_price=10&max_price=200&min_age=18&max_age=100&gender_pref=none&st=0&kw=' + subject + '&ol=false&z=' + str(zip_code)
https://www.wyzant.com/match/search?sort=1&d=20&utc_offset=-5&min_price=10&max_price=200&min_age=18&max_age=100&gender_pref=none&st=0&kw=english&ol=false&z=78751